In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers, models, losses, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [16]:
def make_model(
                filters = 32,
                dense_inner_size = 64,
                lr = 0.002,
                momentum = 0.8,
                loss = tf.keras.losses.BinaryCrossentropy()
                ):
    inputs = keras.Input(shape=(200, 200, 3))
    
    # Convolutional layers
    cvl_layers = layers.Conv2D(filters=filters, kernel_size=(3, 3), activation='relu')(inputs)
    feature_map = layers.MaxPooling2D(pool_size=(2, 2))(cvl_layers)
    
    # Flatten layer
    vectors = layers.Flatten()(feature_map)
    
    # Dense layers
    dense_layers = layers.Dense(dense_inner_size, activation='relu')(vectors)
    
    # Output layer for binary classification
    outputs = layers.Dense(1, activation='sigmoid')(dense_layers)
    
    # Model construction
    model = keras.Model(inputs, outputs)
    
    # Optimizer and compiling the model
    optimizer = optimizers.SGD(learning_rate=lr, momentum=momentum)
    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy']
    )
    
    return model

In [17]:
model = make_model()

In [18]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 200, 200, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 198, 198, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 313632)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │    20,072,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,073,473 (76.57 MB)

 Trainable params: 20,073,473 (76.57 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
train_gen = ImageDataGenerator(rescale=1./255)


train_ds = train_gen.flow_from_directory(
    './data/train',
    target_size=(200, 200), # Size of the image
    batch_size=20  # How many image at once we need
)
test_gen = ImageDataGenerator(rescale=1./255)
test_ds = test_gen.flow_from_directory(
    './data/test',
    target_size=(200, 200), # Size of the image
    batch_size=20  # How many image at once we need
)

Found 800 images belonging to 2 classes.
Found 201 images belonging to 2 classes.


In [ ]:
model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)